In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%cd ..

/scratch/km817/iREC


In [4]:
import torch
import torch.distributions as dist
import math
import matplotlib.pyplot as plt
import pickle as pkl
from tqdm.notebook import tqdm
from rec.beamsearch.distributions.CodingSampler import CodingSampler
from rec.beamsearch.distributions.VariationalPosterior import VariationalPosterior
from rec.OptimisingVars.VariationalOptimiser import VariationalOptimiser
from rec.beamsearch.samplers.GreedySampling import GreedySampler
from rec.beamsearch.Coders.Encoder_Variational import Encoder as Variational_Encoder
from models.SimpleBayesianLinRegressor import BayesLinRegressor
from rec.utils import kl_estimate_with_mc, compute_variational_posterior, plot_samples_in_2d, plot_running_sum_2d, plot_pairs_of_samples

In [5]:
torch.set_default_tensor_type(torch.DoubleTensor)

In [6]:
def encode_sample(target, omega=5, epsilon=0.,
                  n_empirical_samples=10, seed=0, beamwidth=1, optimising_vars=False, aux_vars=None, dont_run=False):
    
    target = compute_variational_posterior(target)
    encoder = Variational_Encoder(target,
                                  seed,
                                  CodingSampler,
                                  GreedySampler,
                                  VariationalPosterior,
                                  omega,
                                  epsilon=epsilon,
                                  beamwidth=beamwidth
                                  )
    if aux_vars is not None:
        encoder.auxiliary_posterior.coding_sampler.auxiliary_vars = aux_vars
    
    if dont_run:
        return encoder
    else:
        return encoder, *encoder.run_encoder()

In [7]:
def create_blr_problem(dim, seed):
    signal_std = 1e-1
    prior_alpha = 1
    num_training = dim
    torch.set_default_tensor_type(torch.DoubleTensor)
    blr = BayesLinRegressor(prior_mean=torch.zeros(dim),
                            prior_alpha=prior_alpha,
                            signal_std=signal_std,
                            num_targets=2 * num_training,
                            seed=seed,
                            num_train_points=num_training)
    blr.sample_feature_inputs()
    blr.sample_regression_targets()
    blr.posterior_update()
    target = blr.weight_posterior
    return blr, target

In [24]:
def compute_expected_coding_efficiency(kl, epsilon=0.2):
    K = (1 + epsilon) * kl
    return K + torch.log(K + 1) + 1

In [55]:
dim = 50
beamwidth = 5
omega = 5
blr_seed = 1
b, t = create_blr_problem(dim=dim, seed=blr_seed)
num_compressed_samples = 500

In [56]:
prior = dist.MultivariateNormal(loc=torch.zeros_like(t.mean), covariance_matrix=torch.eye(dim))

emp_kl = dist.kl_divergence(t, prior)
var_kl = dist.kl_divergence(compute_variational_posterior(t), prior)
print(f"Emp kl: {emp_kl.item():.2f}, Var kl: {var_kl.item()}")

Emp kl: 195.36, Var kl: 219.0374861579449


In [57]:
torch.set_default_tensor_type(torch.DoubleTensor)
aux_vars = None
compute_params_enc = encode_sample(target=t, dont_run=True)
n_auxiliaries = compute_params_enc.n_auxiliary
print(n_auxiliaries)
kl_q_p = compute_params_enc.total_kl
optimising = VariationalOptimiser(compute_params_enc.target, omega, n_auxiliaries, kl_q_p, n_trajectories=32, total_var=1)
aux_vars = optimising.run_optimiser(epochs=2500, lr=0.1)
pkl.dump(aux_vars, open(f"PickledStuff/BLR_RESULTS_v2/Dim{dim}/optimised_vars_var.pkl", "wb"))

  0%|          | 0/2500 [00:00<?, ?it/s]

44


The mean loss is 7.47436. The mean KL is: 5.19139: 100%|██████████| 2500/2500 [3:05:31<00:00,  4.45s/it]   


In [58]:
epsilons = [-0.3, -0.2, -0.1, 0., 0.1, 0.2, 0.3, 0.4]
determine_epsilons = torch.tensor(epsilons)

In [59]:
compute_expected_coding_efficiency(emp_kl, determine_epsilons)

tensor([142.6772, 162.3458, 181.9989, 201.6396, 221.2705, 240.8931, 260.5087,
        280.1185])

In [60]:
compute_expected_coding_efficiency(var_kl, determine_epsilons)

tensor([159.3653, 181.4018, 203.4227, 225.4313, 247.4299, 269.4203, 291.4038,
        313.3815])

In [ ]:
torch.manual_seed(0)
seeds = torch.randint(low = 0, high = int(1e6), size=(num_compressed_samples,))
for eps in epsilons:
    exp_dict = {}
    exp_dict['seeds'] = seeds.numpy()
    exp_dict['target_mean'] = t.mean.numpy()
    exp_dict['target_covar'] = t.covariance_matrix.numpy()
    exp_dict['compressed_samples'] = []
    exp_dict['compressed_samples_idxs'] = []
    exp_dict['aux_vars'] = aux_vars
    pbar = tqdm(enumerate(seeds), total=num_compressed_samples)
    log_probs = torch.zeros([0])
    for i, s in pbar:
        enc, z, idx = encode_sample(target=t, beamwidth=beamwidth, epsilon=eps, omega=omega, 
                                    seed=s, n_empirical_samples=50, aux_vars=aux_vars)
        idxs_to_transmit = idx[0]
        best_sample = z[0]
        log_probs = torch.cat((log_probs, t.log_prob(best_sample)[None]))
        exp_dict['compressed_samples'].append(best_sample.numpy())
        exp_dict['compressed_samples_idxs'].append(idxs_to_transmit.numpy())
        pbar.set_description(f"Coded sample {i + 1}, has log prob of {t.log_prob(best_sample)}")
    
    print(torch.mean(log_probs))
    with open(f"PickledStuff/BLR_RESULTS_v2/Dim{dim}/Variational_Epsilon{eps}_Beam{beamwidth}_Omega{omega}.pkl", "wb") as f:
        pkl.dump(exp_dict, f)

  0%|          | 0/500 [00:00<?, ?it/s]

tensor(-15811.8763)


  0%|          | 0/500 [00:00<?, ?it/s]

tensor(-8789.5903)


  0%|          | 0/500 [00:00<?, ?it/s]

tensor(-4126.3253)


  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
print('done')